In [1]:
import pandas as pd
import numpy as np

## Read Data

In [2]:
###----- Failure Data
df_fail = pd.read_csv('Failure_data.csv') #csv with compatible col names
df_fail.columns = df_fail.columns.str.upper()

# Mutate columns
df_fail['IMMEDIATE_MAINTENANCE'] = df_fail['BREAKDOWN_INDICATOR'] == 'X'
df_fail['IMMEDIATE_MAINTENANCE'] = df_fail['IMMEDIATE_MAINTENANCE'].astype(int)

# Get failure start/end slots
df_fail['FAILURE_START'] = pd.to_datetime(df_fail['FAILURE_START_DATE'] + ' ' +  df_fail['FAILURE_START_TIME'])
df_fail['FAILURE_END'] = df_fail['FAILURE_START'] + pd.to_timedelta(df_fail['ACTUAL_WORK'], unit='h')

df_fail['FAILURE_START_MIN'] = df_fail['FAILURE_START'].dt.ceil('10min') # reading after failure will be erroneous
df_fail['FAILURE_END_MAX'] = df_fail['FAILURE_END'].dt.floor('10min') # reading before fixing will be erroneous

df_fail = df_fail[['FAILURE_START_MIN', 'FAILURE_END_MAX']]

# List of fail slots
fail_slots = []
for row in np.arange(0, df_fail.shape[0]):
    fail_slots.append(pd.date_range(df_fail['FAILURE_START_MIN'][row], df_fail['FAILURE_END_MAX'][row], freq='10min'))

# List of all 10 min time slots when machine was under 'failed' state 
fail_list = [slot for sublist in fail_slots for slot in sublist]

print('Min Fail time slot--', min(fail_list))
print('Max Fail time slot--', max(fail_list))

Min Fail time slot-- 2015-01-05 01:50:00
Max Fail time slot-- 2016-12-16 14:50:00


In [3]:
###----- Sensor Data
df_sensor = pd.read_csv('Sensor_data.csv', low_memory=False)
df_sensor.columns = df_sensor.columns.str.upper()

# Rename columns
org_Sensor_cols = df_sensor.columns # original for lookup if needed
new_colnm = dict(zip(df_sensor.columns[1:],['S'+ str(x) for x in range(1, df_sensor.shape[1])]))
df_sensor.rename(new_colnm, inplace=True, axis=1)

# Manipulate Columns
# Timestamp
df_sensor['TIMESTAMP'] = pd.to_datetime(df_sensor['TIMESTAMP'])
df_sensor['DATE'] = df_sensor['TIMESTAMP'].dt.date
df_sensor['TIME_SLOT_HR'] = df_sensor['TIMESTAMP'].dt.hour
df_sensor['TIME_SLOT_MIN'] = pd.cut(df_sensor['TIMESTAMP'].dt.minute, bins=[0, 10, 20, 30, 40, 50, 60],labels=[1,2,3,4,5,6], right=False)

In [4]:
# Check if data is present for all time slots as expected
x = df_sensor.groupby(['DATE'])['TIMESTAMP'].nunique()
print(x[x<144])

"""Data is missing for 1 hour on 2 dates. Could be because of daylight saving start.
The dates do not align with actual daylight saving time (maybe dates reshuffled for data security?).
Anyhow, in that case, timestamps should be duplicated for some other 2 dates"""

# Find missing time slots
slot_list = pd.date_range(df_sensor['TIMESTAMP'].min(), df_sensor['TIMESTAMP'].max(), freq='10min')
print('\n','Missing timeslots -- ','\n', slot_list[~slot_list.isin(df_sensor['TIMESTAMP'])])

DATE
2016-03-27    138
2017-03-26    138
2017-04-27      1
Name: TIMESTAMP, dtype: int64

 Missing timeslots --  
 DatetimeIndex(['2016-03-27 01:00:00', '2016-03-27 01:10:00',
               '2016-03-27 01:20:00', '2016-03-27 01:30:00',
               '2016-03-27 01:40:00', '2016-03-27 01:50:00',
               '2017-03-26 01:00:00', '2017-03-26 01:10:00',
               '2017-03-26 01:20:00', '2017-03-26 01:30:00',
               '2017-03-26 01:40:00', '2017-03-26 01:50:00'],
              dtype='datetime64[ns]', freq=None)


In [5]:
print(df_sensor.drop_duplicates().shape) # No duplicate rows present
df_sensor['TIMESTAMP'][df_sensor['TIMESTAMP'].duplicated()]# Duplicate timestamps present
# Seems like dates have been reshuffled although data reflects some inconsistency in timestamp similar to daytme saving
# timestamps wont be tempered with as of now. May revisit those before modeling

(109009, 52)


29820   2015-10-25 01:00:00
29821   2015-10-25 01:10:00
29822   2015-10-25 01:20:00
29823   2015-10-25 01:30:00
29824   2015-10-25 01:40:00
29825   2015-10-25 01:50:00
83244   2016-10-30 01:00:00
83245   2016-10-30 01:10:00
83246   2016-10-30 01:20:00
83247   2016-10-30 01:30:00
83248   2016-10-30 01:40:00
83249   2016-10-30 01:50:00
Name: TIMESTAMP, dtype: datetime64[ns]

In [6]:
# subset columns that are categorical + text
sensor_text_col = ['S2', 'S3', 'S4', 'S6', 'S10', 'S11', 'S12', 'S13', 'S14', 
                  'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21', 'S23', 'S32', 
                  'S33', 'S37', 'S40', 'S43','S46', 'S47', 'S48']

sensor_timecol = ['DATE', 'TIMESTAMP', 'TIME_SLOT_HR', 'TIME_SLOT_MIN']
sensor_numcol = set(df_sensor.columns).difference(sensor_text_col, sensor_timecol)

df_sensor.loc[:, df_sensor.columns.isin(sensor_numcol)].apply(lambda x : x.nunique()) 
# S22, S24, S31 have few unique values. These will act as low variance predictors and may need to be dropped from model

##--- Process numeric columns
# Replace text in numeric columns
df_sensor_num_ = df_sensor[sensor_numcol].apply(lambda x:x.str.upper())

for colnm in df_sensor_num_.columns:
    df_sensor_num_[colnm] = df_sensor_num_[colnm].str.replace("I/O TIMEOUT", '')
    df_sensor_num_[colnm] = df_sensor_num_[colnm].str.replace("BAD INPUT", '')
    df_sensor_num_[colnm] = pd.to_numeric(df_sensor_num_[colnm])

df_sensor_num_.isna().sum() 

"""s35 and s7 have large proportion of nan and replacing nan will change distribution of these variables. 
Replacing nan with mean ,min, max may mask the fact that bad imputs causing nan could be result of failure 
or indication that failure may happen in near future. 
A possible strategy could be to replace nan with 5*max of variable. This spike may be useful for learning process, 
or will atleast indicate that something changed for this observation period.
Rest of the variables have few nans and can also be replaced with 5*max value """ 

df_sensor_num_ = df_sensor_num_.apply(lambda x:x.fillna(x.max()*5))

##--- Process Text based cat columns
df_sensor_text_ = df_sensor[sensor_text_col]
df_sensor_text_ = df_sensor_text_.apply(lambda x:x.str.upper())

df_sensor_text_['S33'] = df_sensor_text_['S33'].str.replace('?','')
df_sensor_text_ = df_sensor_text_.replace({' ':'_', '/':''}, regex=True)

df_sensor_text_ = pd.get_dummies(df_sensor_text_)

## Concatenate all datasets
df_sensor_clean = pd.concat([df_sensor[sensor_timecol], df_sensor_text_, df_sensor_num_], axis=1)

In [8]:
# Add Failure flag to data
df_sensor_clean['FLG_FAIL'] = 0
df_sensor_clean.loc[df_sensor_clean['TIMESTAMP'].isin(fail_list), 'FLG_FAIL'] = 1

In [9]:
# Matched time slots as expected?
df_sensor_clean['FLG_FAIL'].sum() == sum([i>df_sensor_clean['TIMESTAMP'].min() for i in fail_list])

True

In [10]:
# Write data for next step
df_sensor_clean.to_csv('df_model.csv', index=False)